In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
agol = GIS(profile='adoezema_AGO')
print(agol.users.me.username)

adoezema


In [3]:
field_data = agol.content.get(itemid='3b9b6fda7d864e3bb804a7bbcd82f702')

Query unsound records from data

In [45]:
def get_unsound_records(lyr):
    keep_fields = ('GlobalID', 'FACILITYID', 'CIPMOID', 'CONDITION', 'OHMComment', 'OHM_Status',  'last_edited_date', 'last_edited_user','SHAPE')
    lyr_name = lyr.properties.name
    df = lyr.query(where="CONDITION<>'Sound'", out_fields=keep_fields, as_df=True)
    df['Layer'] = lyr_name
    return df

In [58]:
dfs = []
for lyr in field_data.layers:
    lyr_name = lyr.properties.name
    if lyr_name not in ('Miscellaneous Point', 'Neighborhoods'):
        df = get_unsound_records(lyr)
        dfs.append(df)
unsound_df  = dfs[0].append(dfs[1:])